In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms

In [2]:
# Activar optimización en GPU
torch.backends.cudnn.benchmark = True

# Configurar GPU o CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

# Transformaciones para CIFAR-10 (ajustado a 224x224 para ResNet-50)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

Usando: cuda


In [3]:
# Cargar datos de entrenamiento y prueba (ajustando batch_size según GPU)
batch_size = 128 

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

# Clases de CIFAR-10
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Cargar modelo ResNet-50 preentrenado y modificar la última capa para 10 clases
resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, 10)
resnet50 = resnet50.to(device)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


c:\Users\artur\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\artur\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
# Definir función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(resnet50.parameters(), lr=0.001)

# Entrenamiento con GPU y Mixed Precision (AMP)
num_epochs = 10
scaler = torch.cuda.amp.GradScaler()  # Para aceleración con AMP

for epoch in range(num_epochs):
    running_loss = 0.0
    resnet50.train()  # Modo entrenamiento

    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)

        # Reiniciar gradientes
        optimizer.zero_grad()

        # Usar AMP para entrenar más rápido en GPU
        with torch.cuda.amp.autocast():
            outputs = resnet50(inputs)
            loss = criterion(outputs, labels)

        # Escalar el gradiente para evitar underflow
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item()

    print(f"🔹 Época {epoch+1}, Pérdida: {running_loss / len(trainloader):.4f}")

print("Entrenamiento completado")

C:\Users\artur\AppData\Local\Temp\ipykernel_17056\1017043375.py:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()  # Para aceleración con AMP
C:\Users\artur\AppData\Local\Temp\ipykernel_17056\1017043375.py:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


🔹 Época 1, Pérdida: 0.6155
🔹 Época 2, Pérdida: 0.3621
🔹 Época 3, Pérdida: 0.2769
🔹 Época 4, Pérdida: 0.2073
🔹 Época 5, Pérdida: 0.1701
🔹 Época 6, Pérdida: 0.1447
🔹 Época 7, Pérdida: 0.1104
🔹 Época 8, Pérdida: 0.0930
🔹 Época 9, Pérdida: 0.0797
🔹 Época 10, Pérdida: 0.0743
Entrenamiento completado


In [5]:
# Guardar el modelo entrenado
torch.save(resnet50.state_dict(), "CIFAR10_resnet50.pth")
print("Modelo guardado correctamente")

Modelo guardado correctamente
